In [1]:
import pandas as pd

# Load the data
df = pd.read_csv('/content/Unit_1_Project_Dataset (1).csv')

# Check basic info
print(df.info())




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7833 entries, 0 to 7832
Data columns (total 34 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   index                        7833 non-null   int64  
 1   host_id                      7833 non-null   int64  
 2   host_name                    7833 non-null   object 
 3   host_since_year              7833 non-null   int64  
 4   host_since_anniversary       7833 non-null   object 
 5   id                           7833 non-null   int64  
 6   neighbourhood_cleansed       7833 non-null   object 
 7   city                         7833 non-null   object 
 8   state                        7825 non-null   object 
 9   zipcode                      7660 non-null   object 
 10  country                      7833 non-null   object 
 11  latitude                     7833 non-null   float64
 12  longitude                    7833 non-null   float64
 13  property_type     

In [2]:
# Drop unnecessary columns
df = df.drop(['index', 'id', 'host_id', 'host_name', 'host_since_anniversary'], axis=1)



In [3]:
# Handle missing values (example: fill missing 'bedrooms' with median)
df['bedrooms'].fillna(df['bedrooms'].median(), inplace=True)
df['bathrooms'].fillna(df['bathrooms'].median(), inplace=True)
df['beds'].fillna(df['beds'].median(), inplace=True)



<ipython-input-3-d34c352dab79>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['bedrooms'].fillna(df['bedrooms'].median(), inplace=True)
<ipython-input-3-d34c352dab79>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

In [4]:
# Drop rows where 'price' or 'city' is missing (important fields)
df.dropna(subset=['price', 'city'], inplace=True)



In [5]:
# Fill missing review scores with the average
review_cols = [col for col in df.columns if 'review_scores' in col]
for col in review_cols:
    df[col].fillna(df[col].mean(), inplace=True)

# Final check
print(df.isnull().sum())

host_since_year                  0
neighbourhood_cleansed           0
city                             0
state                            8
zipcode                        173
country                          0
latitude                         0
longitude                        0
property_type                    0
room_type                        0
accommodates                     0
bathrooms                        0
bedrooms                         0
beds                             0
bed_type                         0
price                            0
guests_included                  0
extra_people                     0
minimum_nights                   0
host_response_time             732
host_response_rate             732
number_of_reviews                0
review_scores_rating             0
review_scores_accuracy           0
review_scores_cleanliness        0
review_scores_checkin            0
review_scores_communication      0
review_scores_location           0
review_scores_value 

<ipython-input-5-99cac8e792db>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


In [6]:
# Average price by city
city_price = df.groupby('city')['price'].mean().sort_values(ascending=False)
print(city_price)



city
WZ                                        350.000000
Amsterdam, Rivierenbuurt, Scheldebuurt    180.000000
Nederland                                 160.000000
Amsterdam                                 155.000000
EC Amsterdam                              150.000000
Landsmeer                                 149.000000
Amsterdam Netherlands                     140.000000
Halfweg                                   139.000000
Duivendrecht                              131.666667
North Holland                             130.000000
Амстердам                                 130.000000
Amsterdam                                 129.661127
De Pijp                                   127.500000
Watergraafsmeer                           126.000000
Slotervaart                               114.500000
Jordaan                                   110.846154
Badhoevedorp                              110.000000
Bos en Lommer                             100.000000
Amsterdam, De Baarsjes                   

In [7]:

# Average price by property type
property_price = df.groupby('property_type')['price'].mean().sort_values(ascending=False)
print(property_price)



property_type
Villa              221.750000
Other              180.724138
Boat               179.758410
House              163.443038
Loft               140.714286
Apartment          123.970860
Bed & Breakfast    100.627027
Treehouse          100.000000
Chalet              85.000000
Cabin               77.166667
Yurt                72.000000
Earth House         65.000000
Dorm                47.500000
Hut                 45.000000
Camper/RV           43.181818
Name: price, dtype: float64


In [8]:
# Correlation between price and reviews
correlation = df[['price', 'review_scores_rating']].corr()
print(correlation)

                         price  review_scores_rating
price                 1.000000              0.042908
review_scores_rating  0.042908              1.000000


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error



In [10]:
# Select features and target
features = df[['city', 'property_type', 'bedrooms', 'bathrooms', 'beds', 'review_scores_rating']]
target = df['price']



In [11]:
# Encode text columns
le_city = LabelEncoder()
features['city'] = le_city.fit_transform(features['city'])

le_property = LabelEncoder()
features['property_type'] = le_property.fit_transform(features['property_type'])



<ipython-input-11-e99e6b6e05ac>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['city'] = le_city.fit_transform(features['city'])
<ipython-input-11-e99e6b6e05ac>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['property_type'] = le_property.fit_transform(features['property_type'])


In [17]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Model
model = LinearRegression()
model.fit(X_train, y_train)



LinearRegression()

In [13]:
# Prediction and evaluation
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 3909.3763266052265


In [18]:
# Save the final dataframe for Tableau
features['predicted_price'] = model.predict(features)
features['actual_price'] = target

features.to_csv('airbnb_pricing_for_tableau.csv', index=False)

<ipython-input-18-f230aa82ddaf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['predicted_price'] = model.predict(features)
<ipython-input-18-f230aa82ddaf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['actual_price'] = target


In [26]:
!pip install seaborn plotly


In [27]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px


In [28]:
# Load data
df = pd.read_csv('airbnb_pricing_for_tableau.csv')
df.head()


,city,property_type,bedrooms,bathrooms,beds,review_scores_rating,predicted_price,actual_price
0,1,0,2.0,2.0,2.0,98.0,167.904755,130
1,1,0,1.0,1.0,2.0,97.0,116.463508,59
2,1,0,1.0,1.0,1.0,92.0,103.399414,95
3,1,0,1.0,1.0,1.0,97.0,106.635647,100
4,1,0,2.0,1.0,2.0,100.0,150.137338,250


In [30]:
import plotly.express as px

# Scatter plot
fig = px.scatter(
    df,
    x="review_scores_rating",
    y="predicted_price",
    color="city",
    hover_data=['property_type', 'bedrooms', 'bathrooms'],
    title="Price vs Review Scores by City"
)

# Update with correct dropdown
fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": str(city),
                    "method": "update",
                    "args": [{"visible": [df['city'] == city]}]
                }
                for city in df['city'].unique()
            ],
            "direction": "down"
        }
    ]
)

fig.show()


In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Example features and target
X = df[['review_scores_rating', 'bedrooms', 'bathrooms']]
y = df['predicted_price']

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Model accuracy
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)


R2 Score: 0.9317940682259636
